In [18]:
## SET UP AZUREML DETAILS
# imports
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Workspace, Environment, Experiment, Dataset, ScriptRunConfig\

# set up workspace
config_path = '../utils/config.json'
tenant_id = '72f988bf-86f1-41af-91ab-2d7cd011db47'  # this is outputted post `az login`
interactive_auth = InteractiveLoginAuthentication(tenant_id=tenant_id)  # create log-in object
ws = Workspace.from_config(path=config_path, auth=interactive_auth)  # link workspace


Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
Performing interactive authentication. Please follow the instructions on the terminal.
You have logged in. Now let us find all the subscriptions to which you have access...
Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
Interactive authentication successfully completed.
Performing interactive authentication. Please follow the instructions on the terminal.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


AuthenticationException: AuthenticationException:
	Message: Could not retrieve user token. Please run 'az login'
	InnerException It is required that you pass in a value for the "algorithms" argument when calling decode().
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "inner_error": {
            "code": "Authentication"
        },
        "message": "Could not retrieve user token. Please run 'az login'"
    }
}

In [ ]:
# set up environment
# - obtain environment.yml from `conda env export > environment.yml`
env_name = 'SampleEnv'
env_path = '../utils/environment.yml'
env = Environment.from_conda_specification(name=env_name, file_path=env_path)

# set up experiment
experiment_name = 'IsolationForestSample'
exp = Experiment(workspace=ws, name=experiment_name)

# set up dataset
dataset_path = 'http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz'
ds = Dataset.File.from_files(dataset_path)

# set up run
src_dir = '../src'
src_name = 'azure_isolation_forest.py'
compute_name = 'WorkspaceCompute'
arguments = ['--data-path', ds.as_mount()]
src = ScriptRunConfig(source_directory=src_dir, script=src_name, compute_target=compute_name,
                      environment=env, arguments=arguments)

In [ ]:
## SUBMIT THE RUN
from azureml.widgets import RunDetails

run = exp.submit(src)  # submit it to the azureml platform
RunDetails(run).show()  # monitor the steps

In [ ]:
# show the outputs
run.wait_for_completion(show_output=True)